In [1]:
import pandas as pd

# Use read_excel instead of read_csv for .xlsx files
data = pd.read_excel('/content/IT Books Names and Descriptions.xlsx')
data.columns = data.columns.str.strip().str.lower()  # Clean column names

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# Lista de stopwords em português
stop_words = stopwords.words('english')

In [4]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(data['description'])

# Calcular a similaridade do cosseno entre os filmes
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [5]:
# Remover sinopses vazias ou muito curtas
data = data.dropna(subset=['description'])
data = data[data['description'].str.len() > 30].reset_index(drop=True)

# Escolher filme de referência (por título exato ou índice)
filme_referencia = data[data['book_name'].str.contains("Game Development with GameMaker Studio 2", case=False)].iloc[0]

# Vetorização TF-IDF
vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2, ngram_range=(1, 2))
tfidf_matrix = vectorizer.fit_transform(data['description'])

# Índice do filme de referência
ref_index = data[data['book_name'] == filme_referencia['book_name']].index[0]

# Calcular similaridade
cosine_sim = cosine_similarity(tfidf_matrix[ref_index], tfidf_matrix).flatten()

# Gerar lista de similaridade (excluindo ele mesmo)
similarities = [
    (data.iloc[i]['book_name'], cosine_sim[i])
    for i in range(len(data)) if i != ref_index and cosine_sim[i] > 0
]

# Ordenar por similaridade
similarities.sort(key=lambda x: x[1], reverse=True)

# Mostrar top 10 resultados
print(f"Filmes mais similares a '{filme_referencia['book_name']}':")
for title, sim in similarities[:10]:
    print(f"{title}: Similaridade = {sim:.4f}")

Filmes mais similares a 'Game Development with GameMaker Studio 2':
GameMaker Essentials: Similaridade = 0.3714
Beginning Game Development with Amazon Lumberyard: Similaridade = 0.3056
HTML5 Game Development from the Ground Up with Construct 2: Similaridade = 0.2945
C++ Game Development Cookbook: Similaridade = 0.2922
Learning Windows 8 Game Development: Similaridade = 0.2886
The Game Jam Survival Guide: Similaridade = 0.2863
Make a 2D Arcade Game in a Weekend: With Unity: Similaridade = 0.2834
iOS Game Development By Example: Similaridade = 0.2811
Mastering Cocos2d Game Development: Similaridade = 0.2801
GameMaker: Studio For Dummies: Similaridade = 0.2759
